In [ ]:
!pip install -q tensorflow-probability

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow as tf

In [ ]:
class VectorQuantizer(layers.Layer):
  def __init__(self, num_embeddings, embedding_dim, beta=0.25, **kwargs):
    super().__init__(**kwargs)
    self.embedding_dim = embedding_dim
    self.num_embeddings = num_embeddings

    # beta best kept betweek [0.25, 2]
    self.beta = beta


    # init embeddings
    w_init = tf.random_uniform_initializer()
    self.embeddings = tf.Variable(
        initial_value = w_init(
            shape=(self.embedding_dim, self.num_embeddings), dtype='float32'
        ),
        trainable=True,
        name='embedding_vqvae',
    )

  def call(self, x):
    input_shape = tf.shape(x)
    flattened = tf.reshape(x, [-1, self.embedding_dim])

    # Quantization
    encoding_indices = self.get_code_indices(flattened)
    encodings = tf.one_hot(encoding_indices, self.num_embeddings)
    quantized = tf.matmul(encodings, self.embeddings, transpose_b=True)

    quantized = tf.reshape(quantized, input_shape)


    commitment_loss = tf.reduce_mean((tf.stop_gradient(quantized) - x) ** 2)
    codebook_loss = tf.reduce_mean((quantized - tf.stop_gradient(x)) ** 2)
    self.add_loss(self.beta * commitment_loss + codebook_loss)

    quantized = x + tf.stop_gradient(quantized - x)

    return quantized


  def get_code_indices(self, flattened_inputs):

    similarity = tf.matmul(flattened_inputs, self.embeddings)
    distances = (
        tf.reduce_sum(flattened_inputs ** 2, axis=1, keepdims=True)
        + tf.reduce_sum(self.embeddings ** 2, axis = 0)
        - 2 * similarity
    )

    encoding_indices = tf.argmin(distances, axis=1)

    return encoding_indices


In [ ]:
def get_encoder(latent_dim=16):
    encoder_inputs = keras.Input(shape=(16, 16, 18))
    x = layers.Conv2D(16, 3, activation="relu", padding='same')(encoder_inputs)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(32, 3, activation="relu", padding='same')(x)
    x = layers.MaxPool2D()(x)
    encoder_outputs = layers.Conv2D(latent_dim, 1, padding="same")(x)
    return keras.Model(encoder_inputs, encoder_outputs, name="encoder")


def get_decoder(latent_dim=16):
    latent_inputs = keras.Input(shape=get_encoder(latent_dim).output.shape[1:])
    x = layers.UpSampling2D()(latent_inputs)
    x = layers.Conv2DTranspose(32, 3, activation="relu", padding='same')(x)
    x = layers.UpSampling2D()(x)
    x = layers.Conv2DTranspose(16, 3, activation="relu", padding='same')(x)
    decoder_outputs = layers.Conv2DTranspose(18, 3, padding="same")(x)
    return keras.Model(latent_inputs, decoder_outputs, name="decoder")

In [ ]:
def get_vqvae(latent_dim=16, num_embeddings=64):

    vq_layer = VectorQuantizer(num_embeddings, latent_dim, name="vector_quantizer")
    #print(vq_layer.embeddings[:])
    encoder = get_encoder(latent_dim)
    decoder = get_decoder(latent_dim)
    inputs = keras.Input(shape=(16, 16, 18))
    encoder_outputs = encoder(inputs)
    quantized_latents = vq_layer(encoder_outputs)
    reconstructions = decoder(quantized_latents)
    return keras.Model(inputs, reconstructions, name="vq_vae")


get_vqvae().summary()

Model: "vq_vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 16, 16, 18)]      0         
                                                                 
 encoder (Functional)        (None, 4, 4, 16)          7776      
                                                                 
 vector_quantizer (VectorQua  (None, 4, 4, 16)         1024      
 ntizer)                                                         
                                                                 
 decoder (Functional)        (None, 16, 16, 18)        11874     
                                                                 
Total params: 20,674
Trainable params: 20,674
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class VQVAETrainer(keras.models.Model):
  def __init__(self, train_variance, latent_dim = 16, num_embeddings=64, **kwargs):
    super(VQVAETrainer, self).__init__(**kwargs)
    self.train_variance = train_variance
    self.latent_dim = latent_dim
    self.num_embeddings = num_embeddings

    self.vqvae= get_vqvae(self.latent_dim, self.num_embeddings)

    self.total_loss_tracker = keras.metrics.Mean(name='total_loss')
    self.reconstruction_loss_tracker = keras.metrics.Mean(
        name='reconstruction_loss'
    )
    self.vq_loss_tracker = keras.metrics.Mean(name='vq_loss')

  @property
  def metrics(self):
    return [
        self.total_loss_tracker,
        self.reconstruction_loss_tracker,
        self.vq_loss_tracker,
    ]

  def train_step(self, x):
    with tf.GradientTape() as tape:
      reconstruction = self.vqvae(x)

      reconstruction_loss = (
          tf.reduce_mean((x - reconstruction) ** 2) / self.train_variance
      )
      total_loss = reconstruction_loss + sum(self.vqvae.losses)

    grads = tape.gradient(total_loss, self.vqvae.trainable_variables)
    self.optimizer.apply_gradients(zip(grads, self.vqvae.trainable_variables))

    self.total_loss_tracker.update_state(total_loss)
    self.reconstruction_loss_tracker.update_state(reconstruction_loss)
    self.vq_loss_tracker.update_state(sum(self.vqvae.losses))


    return {
        'loss': self.total_loss_tracker.result(),
        'reconstruction_loss': self.reconstruction_loss_tracker.result(),
        'vqvae_loss': self.vq_loss_tracker.result(),
    }

In [ ]:
# get vglc data

!git clone https://github.com/TheVGLC/TheVGLC.git

import os




def text_2_mat(s):
  mat = []
  for row in s.split('\n'):
    temp = []
    for item in row:
      temp.append(item)

    mat.append(temp)

  return mat


def mat_2_text(arr):
  text = ''
  for row in arr:
    for item in row:
      text += item
    text += '\n'
  return text[:-1]


def get_chunks(mat):

  chunks = []

  if len(mat.shape) != 2:
    raise ValueError

  smaller_axis = np.argmin(mat.shape)
  smaller_range = np.min(mat.shape)

  if smaller_axis == 0:
    mat = np.transpose(mat, (1,0))

  for i in range(len(mat) - smaller_range):
    chunk = mat[i:i+smaller_range, :]

    if smaller_axis == 0:
      chunk = np.transpose(chunk, (1,0))

    chunks.append(chunk)

  return chunks



def one_hot(x):

  new_x = np.zeros(list(x.shape) + [len(replacement_table)])
  for i in range(x.shape[0]):
    for j in range(x.shape[1]):
      new_x[i, j, :] = replacement_table[x[i,j]]

  return new_x



def chunk_2_text(chunk):
  mat = np.zeros(chunk.shape[:-1]).astype('str')
  for i in range(chunk.shape[0]):
    for j in range(chunk.shape[1]):
      mat[i,j] = replacement_table_inverse[np.argmax(chunk[i,j,:])]

  return mat_2_text(mat)



def print_side_by_side(chunk_1, chunk_2):
  mat_1 = np.zeros(chunk_1.shape[:-1]).astype('str')
  for i in range(chunk_1.shape[0]):
    for j in range(chunk_1.shape[1]):
      mat_1[i,j] = replacement_table_inverse[np.argmax(chunk_1[i,j,:])]

  mat_2 = np.zeros(chunk_2.shape[:-1]).astype('str')
  for i in range(chunk_2.shape[0]):
    for j in range(chunk_2.shape[1]):
      mat_2[i,j] = replacement_table_inverse[np.argmax(chunk_2[i,j,:])]

  mat_mid = np.full((chunk_1.shape[0], chunk_1.shape[1]), ' ').astype('str')


  return mat_2_text(np.concatenate((mat_1, mat_mid, mat_2), axis=1))

def get_smb_data():
  path_to_data = '/content/TheVGLC/Super Mario Bros/Processed/'

  data_chunks = []

  for file_name in os.listdir(path_to_data):
    with open(path_to_data + file_name, 'r') as f:
      s = f.read()
      mat1 = np.array(text_2_mat(s)[:-1]).astype('str')
      mat2 = np.full((2, mat1.shape[1]), fill_value='-')
      mat = np.concatenate((mat2, mat1), axis=0)
      data_chunks += get_chunks(mat)

  one_hot_chunks = []
  for chunk in data_chunks:
    one_hot_chunks.append(one_hot(chunk))


  print('Extracted {} data points with shape {}'.format(len(one_hot_chunks), one_hot_chunks[0].shape))


  raw_data = np.stack(one_hot_chunks)
  np.random.shuffle(raw_data)


  print('shape:{}  min:{}  max:{}  dtype:{}'.format(raw_data.shape, raw_data.min(), raw_data.max(), raw_data.dtype))

  return raw_data


fatal: destination path 'TheVGLC' already exists and is not an empty directory.


In [ ]:
def get_ki_data():
  path_to_data = '/content/TheVGLC/Kid Icarus/Processed/'

  data_chunks = []

  for file_name in os.listdir(path_to_data):
    with open(path_to_data + file_name, 'r') as f:
      s = f.read()
      mat = np.array(text_2_mat(s)[:-1]).astype('str')
      data_chunks += get_chunks(mat)

  one_hot_chunks = []
  for chunk in data_chunks:
    one_hot_chunks.append(one_hot(chunk))


  print('Extracted {} data points with shape {}'.format(len(one_hot_chunks), one_hot_chunks[0].shape))


  raw_data = np.stack(one_hot_chunks)
  np.random.shuffle(raw_data)


  print('shape:{}  min:{}  max:{}  dtype:{}'.format(raw_data.shape, raw_data.min(), raw_data.max(), raw_data.dtype))

  return raw_data


In [ ]:
'''
we do not need all these==> just elements of map 1-1
'''

replacement_table = {
         "X": None,
        "S" : None,
        "-" : None,
        "?" : None,
        "Q" : None,
        "E" : None,
        "<" : None,
        ">" : None,
        "[" : None,
        "]" : None,
        "o" : None,
        "B" : None,
        "b" : None,
        "#" : None,
        "D" : None,
        "H" : None,
        "M" : None,
        "T" : None,
        }



for i, v in enumerate(replacement_table):
  replacement_table[v] = np.array([0] * i + [1] + [0] * (len(replacement_table) - i - 1))

replacement_table_inverse = list(replacement_table.keys())

In [ ]:
print(replacement_table)
print(replacement_table_inverse)

{'X': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'S': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), '-': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), '?': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'Q': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'E': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), '<': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), '>': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), '[': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ']': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]), 'o': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), 'B': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), 'b': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]), '#': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]), 'D': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
smb_data = get_smb_data()
ki_data = get_ki_data()

'''
we need to have the same shape
'''

Extracted 2683 data points with shape (16, 16, 18)
shape:(2683, 16, 16, 18)  min:0.0  max:1.0  dtype:float64
Extracted 1153 data points with shape (16, 16, 18)
shape:(1153, 16, 16, 18)  min:0.0  max:1.0  dtype:float64


'\nwe need to have the same shape\n'

In [ ]:
raw_data = np.concatenate([smb_data, ki_data])
np.random.shuffle(raw_data)

In [ ]:
x_train = raw_data[:]
x_test = x_train

In [ ]:
x_train.shape

(3836, 16, 16, 18)

In [ ]:
x_test.shape

(3836, 16, 16, 18)

In [ ]:
data_variance = x_train.var()

In [ ]:
smb_data.shape

(2683, 16, 16, 18)

In [ ]:
smb_data_indexes=np.random.choice(len(smb_data),100)
smb_data_samples = smb_data[[smb_data_indexes]]

<ipython-input-18-90500cfa9d43>:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  smb_data_samples = smb_data[[smb_data_indexes]]


In [ ]:
smb_data_samples.shape

(100, 16, 16, 18)

In [ ]:
ki_data_indexes=np.random.choice(len(ki_data),100)
ki_data_samples = ki_data[[ki_data_indexes]]

<ipython-input-20-555e3f49e4c8>:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  ki_data_samples = ki_data[[ki_data_indexes]]


In [ ]:
ki_data_samples.shape

(100, 16, 16, 18)

In [ ]:
init_data = np.concatenate([smb_data_samples,ki_data_samples])
np.random.shuffle(init_data)

In [ ]:
data_variance = init_data.var()

In [ ]:
vqvae_trainer = VQVAETrainer(data_variance, latent_dim=16, num_embeddings=64)
vqvae_trainer.compile(optimizer=keras.optimizers.Adam())
vqvae_trainer.fit(x_train, epochs=10, batch_size=32)
vqvae_trainer.vqvae.save_weights('model_new1.h5')
vq_layer = vqvae_trainer.vqvae.layers[2]
emb1=vq_layer.embeddings
a = np.array(emb1)
np.save('codebook_new1',a)

Epoch 1/10
120/120 [==============================] - 7s 55ms/step - loss: 0.7417 - reconstruction_loss: 0.3644 - vqvae_loss: 0.2164
Epoch 2/10
120/120 [==============================] - 7s 56ms/step - loss: 0.4893 - reconstruction_loss: 0.2467 - vqvae_loss: 0.2383
Epoch 3/10
120/120 [==============================] - 7s 55ms/step - loss: 0.6159 - reconstruction_loss: 0.2238 - vqvae_loss: 0.3869
Epoch 4/10
120/120 [==============================] - 7s 55ms/step - loss: 0.3992 - reconstruction_loss: 0.1977 - vqvae_loss: 0.2005
Epoch 5/10
120/120 [==============================] - 7s 55ms/step - loss: 0.3903 - reconstruction_loss: 0.1838 - vqvae_loss: 0.2027
Epoch 6/10
120/120 [==============================] - 7s 56ms/step - loss: 0.3366 - reconstruction_loss: 0.1704 - vqvae_loss: 0.1587
Epoch 7/10
120/120 [==============================] - 7s 55ms/step - loss: 0.2866 - reconstruction_loss: 0.1629 - vqvae_loss: 0.1237
Epoch 8/10
120/120 [==============================] - 8s 66ms/step - 

In [ ]:
trained_vqvae_model = vqvae_trainer.vqvae

input = init_data

input_layer = vqvae_trainer.vqvae.layers[0]
encoder_layer = vqvae_trainer.vqvae.layers[1]
vq_layer = vqvae_trainer.vqvae.layers[2]
decoder_layer = vqvae_trainer.vqvae.layers[3]


input_layer_x = input_layer(input)
encoder_layer_x = encoder_layer(input_layer_x)

vq_layer_x = vq_layer(encoder_layer_x)     ### For GA we need this !

decoder_layer_x = decoder_layer(vq_layer_x)
direct_output = trained_vqvae_model.predict(input) ## decoder output

7/7 [==============================] - 1s 18ms/step


In [ ]:
init_population=np.array(vq_layer_x)
init_population_list=[]

for i in range(len(init_population)):
  init_population_list.append(init_population[i].reshape(-1, 16).tolist())

print(init_population[0].shape)

print(init_population_list[:3])
print(len(init_population_list[0][0]))

## create the codebook a[:][i]
a = a.T

print(a.shape)

if a.shape == (64,16):
  for i in init_population_list:
    for j in i:
      if j not in a:
        print(j)
        print(len(j))
        print(a[0])
        print(a[0].shape)
else:
  print("run again")

#QUESTIONS
## HOW CODEBOOK WAS GENERATED IN COLUMN MANNER?
## WHAT IS CODEBOOK

(4, 4, 16)
[[[0.3613823354244232, 0.20851649343967438, 0.10268021374940872, -0.5401704907417297, -0.20404848456382751, -0.5413349866867065, -0.6137734651565552, 0.36509108543395996, 0.46239104866981506, -0.29573532938957214, -0.4568139314651489, 0.14603693783283234, -0.1900632381439209, 0.46691274642944336, -0.43053674697875977, -0.06848406046628952], [0.5815469622612, 0.8193275332450867, -0.043671589344739914, -0.23612618446350098, 0.1987346112728119, -0.370801717042923, -0.4949207007884979, 0.6815537214279175, 0.5220808386802673, -0.5869254469871521, -0.2245737463235855, 0.16372837126255035, -0.21941831707954407, 0.2997949421405792, -0.6521328687667847, -0.18093407154083252], [0.5815469622612, 0.8193275332450867, -0.04367157816886902, -0.23612618446350098, 0.1987346112728119, -0.370801717042923, -0.4949207007884979, 0.6815537214279175, 0.5220808386802673, -0.5869254469871521, -0.2245737463235855, 0.16372835636138916, -0.21941831707954407, 0.2997949421405792, -0.6521328687667847, -0.1

###Genetic algorithm part

In [ ]:
import numpy as np
#import Data_Pipeline_Test as dpt
import sys
import os
import cv2
import glob
from PIL import Image #Need to import this to do image editing


#### Visualizer
def pad(array, shape):
    print(array.shape, shape)
    board = np.zeros(shape)
    # area = [0: array.shape[0],0: array.shape[1],0: array.shape[2]]
    board[0: array.shape[0],0: array.shape[1],0: array.shape[2]] = array
    return board

char_2_tile_path = {
         "X": '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/X.png',
        "S" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/S.png',
        "-" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/-.png',
        "?" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/Q.png',
        "Q" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/Q.png',
        "E" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/E.png',
        "<" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/PTL.png',
        ">" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/PTR.png',
        "[" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/[.png',
        "]" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/].png',
        "o" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/o.png',
        "B" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/0.png',
        "b" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/0.png',
        "#" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/#.png',
        "D" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/D.png',
        "H" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/H.png',
        "M" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/M.png',
        "T" : '/Users/mohammadrezahami/Documents/University/PCG/Project/Code/sprites/T.png',
        }

def get_tiles():
  char_2_tile = {}
  for char, path in char_2_tile_path.items():
    img = cv2.imread(path)

    char_2_tile[char] = pad(img, shape=(16,16,3))

  return char_2_tile

char_2_tile = get_tiles()


def visualize_ga(chunk):
  img = np.zeros(shape=(14 * 16, 16 * 16, 3))

  row_idx = 0
  col_idx = 0

  for row in chunk:
    for char in row:
      img[row_idx:row_idx+16, col_idx:col_idx+16, :] = char_2_tile[char]
      col_idx += 16
    row_idx += 16
    col_idx = 0

  return img


def visualize(individual):

    #Load the set of all sprites
    sprites = {}
    for filename in glob.glob(os.path.join(os.getcwd(), "sprites", "*.png")):
        im = Image.open(filename)
        splits = filename.split("/")
        name = splits[-1][:-4]
        sprites[name] = im.convert('RGBA')

    #This gives the mapping between the tile values and the associated sprite
    visualization = {}
    visualization["X"] = "X1"
    visualization["S"] = "S"
    visualization["-"] = "-"
    visualization["?"] = "Q"
    visualization["Q"] = "Q"
    visualization["E"] = "E"
    visualization["<"] = "PTL"
    visualization[">"] = "PTR"
    visualization["["] = "pipe"
    visualization["]"] = "pipe_r"
    visualization["o"] = "o"
    #visulization["B"] = ""
    #visulization["b"] =
    visualization["T"] = "T"
    visualization["M"] = "M"
    visualization["D"] = "D"
    visualization["#"] = "X"
    visualization["H"] = "H"

    # This reads in the level
    # level = {}
    # with open(os.path.join(os.getcwd(), "Generated Levels", "output.txt")) as fp:
    #     y = 0
    #     for line in fp:
    #         level[y] = line
    #         y+=1

    #Multiply by 18 here as each of the sprites is 14*16
    image = Image.new("RGB", (14*len(individual[0]), 16*len(individual)), color=(223, 245, 244)) #This creates an initially blank image for the level
    pixels = image.load() #This loads the level image's pixels so we can edit them

    maxY = len(individual)
    maxX = len(individual[0])

    for y in range(0, maxY):
        for x in range(0, maxX):
            imageToUse = None
            if individual[y][x] in visualization.keys():
                imageToUse = sprites[visualization[individual[y][x]]]
                #print(imageToUse)
            #elif individual[y][x]=="X":
                #Rules we've added to ensure the correct sprite is used
                #if y==maxY-2:
                   # imageToUse = sprites["groundTop"]
                # elif y==maxY-1:
                #     #Check if we have a solid tile above this and change which sprite we use if so
                #     if individual[y-1][x]=="X":
                #         imageToUse = sprites["groundBottom"]
                #     else:
                #         imageToUse = sprites["groundTop"]
                # else:
                #     imageToUse = sprites["stair"]

            if not imageToUse == None:
                #If we have a sprite (imageToUse) copy its pixels over
                pixelsToUse = imageToUse.load()
                for x2 in range(0, 13):
                    for y2 in range(0, 15):
                        if pixelsToUse[x2,y2][3]>0:
                            pixels[x*14+x2,y*16+y2] = pixelsToUse[x2,y2][0:-1]
    return image


def mutation(individual, mutation_rate):
    if np.random.random()<mutation_rate:

        ###attempt1
        ## change one cell to soomething random between 1 and 16
        row = np.random.randint(0,len(individual))
        column = np.random.randint(0,len(individual[0]))
        smb_elements = ['S','?','Q','E','<','>','[',']','o','B','b']
        kl_elements = ['T','M','D','H']
        elements = smb_elements + kl_elements
        individual[row][column] = elements[np.random.randint(0,15)]
        return individual

        ###attempt2
        ## swap 2 cells of the individual
        # row_1 = np.random.randint(0,len(individual))
        # column_1 = np.random.randint(0,len(individual[0]))

        # row_2 = np.random.randint(0,len(individual))
        # column_2 = np.random.randint(0,len(individual[0]))

        # while not (row_1==row_2 and column_1==column_2):
        #     row_2 = np.random.randint(0,len(individual))
        #     column_2 = np.random.randint(0,len(individual[0]))
        # tmp = individual[row_1][column_1]
        # individual[row_1][column_1] = individual[row_2][column_2]
        # individual[row_2][column_2] = tmp
        # return individual
    else:
        return individual

def crossover(individual_1, individual_2, crossover_rate):
    child_1 = np.array([[]]).reshape(0,16)
    child_2 = np.array([[]]).reshape(0,16)
    if np.random.random()<crossover_rate:

        # if np.random.randint(0,2):
        if 0:
            ##crossover by row

            rand_row_cut = np.random.randint(0, 13)
            child_1 = np.concatenate((child_1,individual_1[:rand_row_cut+1]))
            #child_1 = individual_1[:rand_row_cut+1]
            child_1 = np.concatenate((child_1,individual_2[rand_row_cut+1:]))
            #child_1 += individual_2[rand_row_cut+1:]


            child_2 = np.concatenate((child_2,individual_1[rand_row_cut+1:]))
            #child_2 = individual_1[rand_row_cut+1:]
            child_2 = np.concatenate((child_2,individual_2[:rand_row_cut+1]))
            #child_2 += individual_2[:rand_row_cut+1]

            return child_1, child_2

        else:
            ##crossover by column
            rand_cul_cut = np.random.randint(0, 14)
            for row in range(len(individual_1)):
                temp1 = np.array([[]]).reshape(0,16)
                temp1 = np.concatenate((individual_1[row][:rand_cul_cut+1],individual_2[row][rand_cul_cut+1:]))
                #child_1=individual_1[row][:rand_cul_cut+1]
                temp1 = np.array([temp1])
                child_1 = np.concatenate((child_1,temp1))
                #child_1[row] += individual_2[row][rand_cul_cut+1:]

                temp2 = np.array([[]]).reshape(0,16)
                temp2 = np.concatenate((individual_1[row][rand_cul_cut+1:],individual_2[row][:rand_cul_cut+1]))
                #child_2.append(individual_1[row][rand_cul_cut+1:])
                #child_2[row] += individual_2[row][:rand_cul_cut+1]
                temp2 = np.array([temp2])
                child_2 = np.concatenate((child_2,temp2))

            return child_1, child_2
    else:
        return child_1, child_2

def evolution(population, population_limit,passable_point,passable_smb_sum,passable_kid_sum, solid_point, solid_smb_sum, solid_kid_sum):
    newpopulation = []
    fitness_population = []
    for individual in population:
        fitness_population.append((fitness(individual, passable_point,passable_smb_sum,passable_kid_sum, solid_point, solid_smb_sum, solid_kid_sum), individual))

    # np.sort(fitness_population,axis=0)

    fitness_population.sort(key=lambda tup: tup[0], reverse=True)
    for i in range(population_limit//5*4):
        newpopulation.append(fitness_population[i][1])

    ## also append some random trash
    inthelist = []
    for i in range(population_limit//5):
        rand_individual = np.random.randint(len(population)//5*4,len(population))


        while rand_individual in inthelist:
            rand_individual = np.random.randint(len(population)//5*4,len(population))

        newpopulation.append(fitness_population[rand_individual][1])
        inthelist.append(rand_individual)

    return newpopulation

def fitness(individual, passable_point,passable_smb_sum,passable_kid_sum, solid_point, solid_smb_sum, solid_kid_sum):
    distribution = {}
    smb_elements = ['X','S','-','?','Q','E','<','>','[',']','o','B','b']
    kl_elements = ['T','M','D','#','H','-']
    elements = smb_elements + kl_elements
    distribution = dpt.distribution_extractor(individual, elements, distribution)


    passable = ['o','-','M','T']
    solid = ['X','S','?','Q', '#','H']
    passable_sum = 0
    solid_sum = 0

    for key in distribution:
        if key in passable:
            passable_sum += distribution[key]
        elif key in solid:
            solid_sum += distribution[key]

    #print("solid",solid_sum)

    #for passable
    fitness_passable = 0
    if passable_smb_sum <= passable_kid_sum:
        if passable_sum >= passable_smb_sum and passable_sum <= passable_point:
            fitness_passable = ((passable_point - passable_sum) / (passable_smb_sum - passable_point)) + 1

        elif passable_sum <= passable_kid_sum and passable_sum >= passable_point:
            fitness_passable = ((passable_point - passable_sum) / (passable_kid_sum - passable_point)) + 1
        else:
            fitness_passable = 0
    else:
        if passable_sum >= passable_kid_sum and passable_sum <= passable_point:
            fitness_passable = ((passable_point - passable_sum) / (passable_kid_sum - passable_point)) + 1

        elif passable_sum <= passable_smb_sum and passable_sum >= passable_point:
            fitness_passable = ((passable_point - passable_sum) / (passable_smb_sum - passable_point)) + 1
        else:
            fitness_passable = 0


    # print("passable_sum:",passable_sum)
    # print("passable_smb:",passable_smb_sum)
    # print("passable_kid:",passable_kid_sum)
    # print("passable_point:",passable_point)
    # print("passable_fitness:",fitness_passable)

    #for solid
    fitness_solid = 0
    if solid_smb_sum <= solid_kid_sum:
        if solid_sum >= solid_smb_sum and solid_sum <= solid_point:
            fitness_solid = ((solid_point - solid_sum) / (solid_smb_sum - solid_point)) + 1

        elif solid_sum <= solid_kid_sum and solid_sum >= solid_point:
            fitness_solid = ((solid_point - solid_sum) / (solid_kid_sum - solid_point)) + 1
        else:
            fitness_solid = 0
    else:
        if solid_sum >= solid_kid_sum and solid_sum <= solid_point:
            fitness_solid = ((solid_point - solid_sum) / (solid_kid_sum - solid_point)) + 1

        elif solid_sum <= solid_smb_sum and solid_sum >= solid_point:
            fitness_solid = ((solid_point - solid_sum) / (solid_smb_sum - solid_point)) + 1
        else:
            fitness_solid = 0

    fitness= (fitness_passable + fitness_solid)/2

    return fitness

def generation(population:list, crossover_rate, mutation_rate, population_limit, passable_point,passable_smb_sum,passable_kid_sum, solid_point, solid_smb_sum, solid_kid_sum):
    ##select parents
    for _ in range(len(population)//3):
        parent_1 = np.random.randint(0,len(population))
        parent_2 = np.random.randint(0,len(population))
        while parent_1 == parent_2:
            parent_2 = np.random.randint(0,len(population))
        child_1, child_2 = crossover(population[parent_1], population[parent_2], crossover_rate)
        ##do crossover
        if child_1.size != 0 and child_2.size != 0:
            population.append(child_1)
            population.append(child_2)

    ##do mutation
    for individual in population:
        individual = mutation(individual, mutation_rate)

    ##do evolution
    population = evolution(population, population_limit, passable_point,passable_smb_sum,passable_kid_sum, solid_point, solid_smb_sum, solid_kid_sum)

    return population


def main():
    num_generation = 500
    population = []
    crossover_rate = 0.8
    mutation_rate = 0.05
    population_limit = 250
    blending_point = 0.33
    mario_address = 'mario-3-3.txt'
    kidicarus_address = 'kidicarus_1.txt'
    passable_point,passable_smb_sum,passable_kid_sum, solid_point, solid_smb_sum, solid_kid_sum = dpt.points_extractor(mario_address, kidicarus_address,blending_point)

    dataset_smb, dataset_kid = dpt.dataset_creator(mario_address,kidicarus_address)

    population = dataset_kid + dataset_smb

    for epoch in range(num_generation):
        print("-----",epoch,"-----")
        population = generation(population, crossover_rate, mutation_rate, population_limit, passable_point, passable_smb_sum, passable_kid_sum, solid_point, solid_smb_sum, solid_kid_sum)

    fitness_population = []
    for individual in population:
        fitness_population.append((fitness(individual, passable_point,passable_smb_sum,passable_kid_sum, solid_point, solid_smb_sum, solid_kid_sum), individual))

    fitness_population.sort(key=lambda tup: tup[0], reverse=True)

    i = 0
    for individual in fitness_population[:50]:
        #print(individual[1],individual[0])
        #print("------------------------------------------")
        #print("------------------------------------------")
        image = visualize_ga(individual[1])
        cv2.imwrite("output"+str(i)+'.png',image)
        # image.save("./outputs/output"+str(i)+".jpeg","JPEG")
        i += 1
        print("output ",str(i),": ",individual[0])

if __name__ == main():
    main()



###### non GA Section

In [ ]:
#raw_data = np.concatenate([smb_data, ki_data])
#np.random.shuffle(raw_data)

In [ ]:
import pickle
vqvae_trainer = VQVAETrainer(data_variance, latent_dim=16, num_embeddings=64)
vqvae_trainer.compile(optimizer=keras.optimizers.Adam())
vqvae_trainer.fit(x_train, epochs=5, batch_size=32) ## 50 , 32



#vq_layer = vqvae_trainer.vqvae.layers[2]
#emb1=vq_layer.embeddings

vqvae_trainer.vqvae.save_weights('model2.h5')


vq_layer = vqvae_trainer.vqvae.layers[2]
emb1=vq_layer.embeddings

# print(type(emb1))
a = np.array(emb1)

np.save('codebook2',a)

# with open("codebook1.txt", 'w') as file:
  # pickle.dump(emb1, file)



Epoch 1/5
120/120 [==============================] - 7s 50ms/step - loss: 0.7361 - reconstruction_loss: 0.3795 - vqvae_loss: 0.1905
Epoch 2/5
120/120 [==============================] - 6s 50ms/step - loss: 0.3495 - reconstruction_loss: 0.2469 - vqvae_loss: 0.0969
Epoch 3/5
120/120 [==============================] - 6s 50ms/step - loss: 0.4638 - reconstruction_loss: 0.2218 - vqvae_loss: 0.2388
Epoch 4/5
120/120 [==============================] - 6s 49ms/step - loss: 0.4570 - reconstruction_loss: 0.2045 - vqvae_loss: 0.2473
Epoch 5/5
120/120 [==============================] - 6s 49ms/step - loss: 0.3406 - reconstruction_loss: 0.1911 - vqvae_loss: 0.1452


In [ ]:
c = np.load('codebook.npy')


model = get_vqvae(latent_dim=16, num_embeddings=64)
model.load_weights('model.h5')

vq_layer = model.layers[2]
emb1=vq_layer.embeddings
emb1=np.array(emb1)


c2 = np.load('codebook2.npy')

model2 = get_vqvae(latent_dim=16, num_embeddings=64)
model2.load_weights('model2.h5')

vq_layer2 = model2.layers[2]
emb2=vq_layer2.embeddings
emb2=np.array(emb2)




emb1==emb2


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
trained_vqvae_model = vqvae_trainer.vqvae

# idx = np.random.choice(len(x_test), 2)

# inpt = x_test[idx]
inpt = x_test[[2796, 139]]

input_layer = vqvae_trainer.vqvae.layers[0]
encoder_layer = vqvae_trainer.vqvae.layers[1]
vq_layer = vqvae_trainer.vqvae.layers[2]
decoder_layer = vqvae_trainer.vqvae.layers[3]


input_layer_x = input_layer(inpt)
encoder_layer_x = encoder_layer(input_layer_x)
vq_layer_x = vq_layer(encoder_layer_x)
decoder_layer_x = decoder_layer(vq_layer_x)

direct_output = trained_vqvae_model.predict(inpt)


# print('direct output\t\t\t decoder output\t\t sample 1')
# print(print_side_by_side(direct_output[0], decoder_layer_x[0]))

# print('direct output\t\t\t decoder output\t\t sample 2')
# print(print_side_by_side(direct_output[1], decoder_layer_x[1]))


xx = vq_layer_x.numpy()

print(xx[0].shape)
A = xx[0].reshape(-1, 16).tolist()
B = xx[1].reshape(-1, 16).tolist()

print(len(A))

A[0] = B[0]

print('MAP ID ', 2796)

for i in range(len(A)):

  xx_decoded = decoder_layer(np.array(A).reshape(1, 4, 4, 16))

  decoded_text = chunk_2_text(xx_decoded[0])
  print(decoded_text)
  #img = visualize(decoded_text)

  ##### img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  #cv2.imwrite('/content/drive/MyDrive/pcgml_temp/visualize/{}.png'.format(i), img)
  print()
  A[i] = B[i]
xx_decoded = decoder_layer(np.array(A).reshape(1, 4, 4, 16))
img = visualize(decoded_text)
cv2.imwrite('/content/drive/MyDrive/pcgml_temp/visualize/{}.png'.format(i+1), img)
print(chunk_2_text(xx_decoded[0]))
print('MAP ID ', 139)


In [ ]:
# trained_vqvae_model = vqvae_trainer.vqvae
idx = np.random.choice(len(x_test), 10)
test_chunks = x_test[idx]
print(idx)
reconstructions_test = trained_vqvae_model.predict(test_chunks)

print('Original\t\t\tReconstructed')
for test_chunk, reconstructed_chunk in zip(test_chunks, reconstructions_test):
  reconstructed_chunk_final = reconstructed_chunk.reshape(16, 16, 18)
  print(print_side_by_side(test_chunk, reconstructed_chunk_final))
  print()



[3187 3446  822 2601  139 1668  274 1019 1256  450]
1/1 [==============================] - 0s 27ms/step
Original			Reconstructed
#----#TTTT--####                -------------###
#--##TTTT--####-                -------------###
#-########----##                #######--------#
#-------------##                #-------------##
#-------------D#                #-------------##
#-------------D#                #-------------##
#-D---------####                #------------###
#-D-----####----                #--------###----
TTTTTTTT--TTTTTT                #---TTTTT-------
----------------                ----------------
---------------#                ----------------
-TTTTTTTT----###                TTTTTTTTT----##-
------------####                ------------####
----------------                ----------------
--------TTT-----                --------TTTT----
----------------                ----------------

----------------                ----------------
----------------                -----

In [ ]:
import cv2

In [ ]:
def pad(array, shape):
    print(array.shape,shape)
    board = np.zeros(shape)
    area = [slice(0, array.shape[dim]) for dim in range(array.ndim)]
    board[area] = array
    return board

In [ ]:
char_2_tile_path = {
         "X": '/content/drive/MyDrive/pcgml_temp/tiles/X.png',
        "S" : '/content/drive/MyDrive/pcgml_temp/tiles/S.png',
        "-" : '/content/drive/MyDrive/pcgml_temp/tiles/-.png',
        "?" : '/content/drive/MyDrive/pcgml_temp/tiles/Q.png',
        "Q" : '/content/drive/MyDrive/pcgml_temp/tiles/Q.png',
        "E" : '/content/drive/MyDrive/pcgml_temp/tiles/E.png',
        "<" : '/content/drive/MyDrive/pcgml_temp/tiles/PTL.png',
        ">" : '/content/drive/MyDrive/pcgml_temp/tiles/PTR.png',
        "[" : '/content/drive/MyDrive/pcgml_temp/tiles/[.png',
        "]" : '/content/drive/MyDrive/pcgml_temp/tiles/].png',
        "o" : '/content/drive/MyDrive/pcgml_temp/tiles/o.png',
        "B" : '/content/drive/MyDrive/pcgml_temp/tiles/0.png',
        "b" : '/content/drive/MyDrive/pcgml_temp/tiles/0.png',
        "#" : '/content/drive/MyDrive/pcgml_temp/tiles/#.png',
        "D" : '/content/drive/MyDrive/pcgml_temp/tiles/D.png',
        "H" : '/content/drive/MyDrive/pcgml_temp/tiles/H.png',
        "M" : '/content/drive/MyDrive/pcgml_temp/tiles/M.png',
        "T" : '/content/drive/MyDrive/pcgml_temp/tiles/T.png',
        }

def get_tiles():
  char_2_tile = {}
  for char, path in char_2_tile_path.items():
    img = cv2.imread(path)

    char_2_tile[char] = pad(img, shape=(16,16,3))

  return char_2_tile

char_2_tile = get_tiles()

NameError: ignored

In [ ]:
def visualize(chunk):
  img = np.zeros(shape=(16 * 16, 16 * 16, 3))

  row_idx = 0
  col_idx = 0

  for char in chunk:
    if char == '\n':
      row_idx += 16
      col_idx = 0
      continue

    img[row_idx:row_idx+16, col_idx:col_idx+16, :] = char_2_tile[char]
    col_idx += 16

  return img

  # image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # cv2.imwrite('/content/drive/MyDrive/pcgml_temp/visualize', image_gray)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sample_input = [
    ['-', '-', 'X', 'S'],
    ['-', '-', '-', '-'],
    ['-', '-', '#', '-'],
    ['-', '-', '#', 'E'],
]

In [ ]:
def pad(array, shape):
    board = np.zeros(shape)
    area = [slice(0, array.shape[dim]) for dim in range(array.ndim)]
    board[area] = array
    return board

char_2_tile_path = {
         "X": '/content/drive/MyDrive/pcgml_temp/tiles/X.png',
        "S" : '/content/drive/MyDrive/pcgml_temp/tiles/S.png',
        "-" : '/content/drive/MyDrive/pcgml_temp/tiles/-.png',
        "?" : '/content/drive/MyDrive/pcgml_temp/tiles/Q.png',
        "Q" : '/content/drive/MyDrive/pcgml_temp/tiles/Q.png',
        "E" : '/content/drive/MyDrive/pcgml_temp/tiles/E.png',
        "<" : '/content/drive/MyDrive/pcgml_temp/tiles/PTL.png',
        ">" : '/content/drive/MyDrive/pcgml_temp/tiles/PTR.png',
        "[" : '/content/drive/MyDrive/pcgml_temp/tiles/[.png',
        "]" : '/content/drive/MyDrive/pcgml_temp/tiles/].png',
        "o" : '/content/drive/MyDrive/pcgml_temp/tiles/o.png',
        "B" : '/content/drive/MyDrive/pcgml_temp/tiles/0.png',
        "b" : '/content/drive/MyDrive/pcgml_temp/tiles/0.png',
        "#" : '/content/drive/MyDrive/pcgml_temp/tiles/#.png',
        "D" : '/content/drive/MyDrive/pcgml_temp/tiles/D.png',
        "H" : '/content/drive/MyDrive/pcgml_temp/tiles/H.png',
        "M" : '/content/drive/MyDrive/pcgml_temp/tiles/M.png',
        "T" : '/content/drive/MyDrive/pcgml_temp/tiles/T.png',
        }

def get_tiles():
  char_2_tile = {}
  for char, path in char_2_tile_path.items():
    img = cv2.imread(path)

    char_2_tile[char] = pad(img, shape=(16,16,3))

  return char_2_tile

char_2_tile = get_tiles()

def visualize_ga(chunk):
  img = np.zeros(shape=(14 * 16, 16 * 16, 3))

  row_idx = 0
  col_idx = 0

  for row in chunk:
    for char in row:
      img[row_idx:row_idx+16, col_idx:col_idx+16, :] = char_2_tile[char]
      col_idx += 16
    row_idx += 16
    col_idx = 0

  return img

(224, 256, 3)